In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name = fn, length = len(uploaded[fn])))

In [0]:
!ls

sample_data


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time
import os 
import psutil

pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)



# DATA PREPROCESSING------------------------------------------------------------
data = pd.read_csv("DARPA1999_week1mon", delimiter=',', header=0, quotechar='"')
tcp_data = data.loc[data['Protocol']=='TCP']

# Gets the next input sequence of packets
def get_sequence(pos, input_data):
  #pos = 0 
  t1 = input_data['Time'].values[pos] # Timestamp of the first packet in the sequence
  src1 = input_data['Source'].values[pos] # Source IP of the first packet in the sequence
  num_packets = 0 # Store the number of packets in the sequence
  sequence = []


  for x in range(pos, pos+320):
    # if packet does not have the same source or destination IP as the source of the first one in the sequence
    if(input_data['Source'].values[x] != src1 and input_data['Destination'].values[x] != src1): 
      break
    # if the current one did not happen within 1 second of the first one in the sequence 
    elif((input_data['Time'].values[x]-t1) > 1): 
      break
    else: 
      row = input_data.values[x]
      rowIndex = input_data.values[x][0]
      #print("ROW INDEX: ", rowIndex)  # Here is the piece of code i put in to check it - Matt 
      sequence.append(row) # Append the sequence with the current packet
      num_packets = num_packets + 1 # Update the number of packets
  
  return num_packets, sequence



In [0]:
# Split into train, test data
train, test = train_test_split(tcp_data, test_size=0.33, random_state=0, shuffle=False)
print("Train: \n", train)
np_data = np.array(train)

# Get all of the sequences
count = 0 # Store the total number of processed packets

all_sequences = pd.DataFrame(data = np_data, columns = ["No.", "Time", "Source IP", "Destination IP", "Protocol", "Length", "Source Port", "Destination Port"])
num_sequences = 0 # Store the total number of sequences

all_sequences_df = pd.DataFrame(columns = ["No.", "Time", "Source IP", "Destination IP", "Protocol", "Length", "Source Port", "Destination Port"])

while count < len(train.index):
  num_packets, sequence = get_sequence(count, train) # Get the next sequence
  count = count + num_packets # Update the number of processed packets
  #all_sequences.append(sequence)
  
  sequence_df = pd.DataFrame(data = sequence, columns = ["No.", "Time", "Source IP", "Destination IP", "Protocol", "Length", "Source Port", "Destination Port"])
  print('Number of packets in sequence: ', num_packets)
  print("Sequence: \n", sequence_df)
  all_sequences_df = all_sequences_df.append(sequence_df)

  num_sequences = num_sequences+1 # Update the number of sequences
  print("Sequence Number: ", num_sequences)
  print("=========================================================\n")
  #print("All Sequences type: ", type(all_sequences))


print("\n\n\n@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print("-----> FINISHED READING SEQUENCES <-----")

In [0]:
#print(all_sequences_df)

#convert dataframe to csv file 
#sequenceCSV = all_sequences_df.to_csv("/content/SequenceCSV.csv")

import psutil 

print(psutil.virtual_memory())

print(psutil.virtual_memory()[0])
print(psutil.virtual_memory()[1])
print(psutil.virtual_memory()[2])

print("Total: ", psutil.virtual_memory()[0] / 1024/1024/1024)

svmem(total=27395407872, available=25382928384, percent=7.3, used=2159669248, free=22743834624, active=1731051520, inactive=2595991552, buffers=75104256, cached=2416799744, shared=868352, slab=181104640)
27395407872
25382961152
7.3
Total:  25.513961791992188
